### PENDIENTE:

### -> 2. Filtro avanzado fechas
### -> 5. Notificación con datos: hospital, nombre concurso, fecha límite

#### CASO DIARIO - 1 entidad especifica:

1. filtros básicos:
- nombre entidad
- tipo: servicio
- año

2. filtros avanzados:
- fecha inicio: fecha actual
- fecha fin: fecha actual

3. colocar captcha para ejecutar busqueda (resultados obtenidos)

4. bucle:
- obtener lista concursos
- guardar info y archivos
    
5. llega notificación con datos: hospital, nombre concurso, fecha límite

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
ruc_prueba = "20153219118"

In [3]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://prod2.seace.gob.pe/seacebus-uiwd-pub/buscadorPublico/buscadorPublico.xhtml")
wait = WebDriverWait(driver, 20)
time.sleep(5)

### 1. FILTRO BÁSICO

#### 1.1. NOMBRE O SIGLA DE ENTIDAD

In [4]:
#clic lupa
lupa_buscar_entidad = wait.until(EC.presence_of_element_located((By.ID, "tbBuscador:idFormBuscarProceso:j_idt28"))).click()
time.sleep(1)
#insertar ruc entidad
elemento_input = driver.find_element(By.CSS_SELECTOR,'[id="tbBuscador:idFormBuscarProceso:txtRucEntidad"][name="tbBuscador:idFormBuscarProceso:txtRucEntidad"]').click()
ActionChains(driver).send_keys_to_element(elemento_input, ruc_prueba).perform()
#buscar ruc entidad
btn_buscar = driver.find_element(By.ID, "tbBuscador:idFormBuscarProceso:btnBuscarEntidad").click()
time.sleep(1)
#capturar nombre entidad
nombre_entidad = driver.find_element(By.ID, "tbBuscador:idFormBuscarProceso:dataTable:0:ajaxEntidad").text
#confirmar entidad
op_confirmar = driver.find_element(By.ID, "tbBuscador:idFormBuscarProceso:dataTable:0:j_idt271").click()  
#confirmación visual
print("LISTO NOMBRE O SIGLA DE ENTIDAD")

LISTO NOMBRE O SIGLA DE ENTIDAD


#### 1.2. OBJETO DE CONTRATACIÓN

In [5]:
#clic contratacion
desp_contratacion = wait.until(EC.presence_of_element_located((By.ID, 'tbBuscador:idFormBuscarProceso:j_idt41_label'))).click()
time.sleep(1)
#clic tipo: servicio
elemento_servicio = driver.find_element(By.CSS_SELECTOR, 'li[data-label="Servicio"]').click()
#confirmación visual
print("LISTO OBJETO DE CONTRATACIÓN")

LISTO OBJETO DE CONTRATACIÓN


#### 1.3. AÑO DE LA CONVOCATORIA

In [6]:
#clic desplegable
desp_convocatoria = wait.until(EC.presence_of_element_located((By.ID, "tbBuscador:idFormBuscarProceso:anioConvocatoria_label"))).click()
time.sleep(1)
#clic año 2023
elemento_2023 = driver.find_element(By.CSS_SELECTOR, 'li[data-label="2023"]').click()
#confirmación visual
print("LISTO OBJETO DE CONTRATACIÓN")

LISTO OBJETO DE CONTRATACIÓN


### 2. FILTRO AVANZADO

- clic opciones avanzadas
- fecha inicio: fecha actual
- fecha fin: fecha actual

-> inicio octubre

-> fin diciembre

### 3. COLOCAR CAPTCHA PARA EJECUTAR BÚSQUEDA

#### 3.1. FUNCIONES

In [7]:
from datetime import datetime

In [8]:
def fn_get_imagen():
    imagen = driver.find_element(By.CSS_SELECTOR, 'img[id="tbBuscador:idFormBuscarProceso:captchaImg"]')
    x, y, width, height = imagen.location['x'], imagen.location['y'], imagen.size['width'], imagen.size['height']
    imagen_pil = Image.open(io.BytesIO(driver.get_screenshot_as_png()))
    imagen_recortada = imagen_pil.crop((x, y, x + width, y + height))
    #nombre = "captcha" + datetime.now().strftime("%d_%m_%Y-%H_%M_%S") + ".png"
    #imagen_recortada.save(nombre)    
    time.sleep(2)
    return imagen_recortada

In [9]:
def fn_lectura_ocr(image):
    grayscale_image = image.convert('L')
    threshold = 50  
    binary_image = grayscale_image.point(lambda p: p < threshold and 255)
    text = pytesseract.image_to_string(binary_image)
    text = text.lower()
    text = text.replace('\n', '')
    text = text.replace("'",'')
    text = text.replace(" ","")
    return text

In [10]:
#devuelve un valor no vacío
def fn_evaluacion(i):
    imagen_recortada = fn_get_imagen()
    time.sleep(2)
    
    lectura_ocr = fn_lectura_ocr(imagen_recortada)
    time.sleep(2)
    print("-------------------------------")
    print("Lectura_ocr_",i+1,": ",lectura_ocr)
    time.sleep(2)
    
    while len(lectura_ocr) != 5:
        
        print ("Lectura incorrecta, nuevo intento...")
        
        i = i + 1
        elemento_captcha = driver.find_element(By.CSS_SELECTOR,'button[id="tbBuscador:idFormBuscarProceso:btnrefreshcaptcha"][name="tbBuscador:idFormBuscarProceso:btnrefreshcaptcha"]').click()
        time.sleep(2)
        
        imagen_recortada = fn_get_imagen()
        time.sleep(2)

        lectura_ocr = fn_lectura_ocr(imagen_recortada)
        time.sleep(2)
        
        print("-------------------------------")
        print("Lectura_ocr_",i+1,": ",lectura_ocr)
        time.sleep(2)
        
    return lectura_ocr,i

#### 3.2. AGREGAR LIB OCR

In [11]:
from PIL import Image
import io

In [12]:
#pip install pytesseract
import os
# Ruta al directorio tessdata
tessdata_dir = "C:\\Program Files\\Tesseract-OCR\\tessdata"
# Configurar la variable de entorno TESSDATA_PREFIX
os.environ["TESSDATA_PREFIX"] = tessdata_dir

In [13]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

#### 3.3. AGREGAR CAPTCHA Y BUSCAR

In [14]:
from selenium.common.exceptions import NoSuchElementException

In [15]:
#ver si hubo error
i=0
elemento = 'x'

while elemento:
    rpta,i = fn_evaluacion(i)
    time.sleep(2)
    
    print("ubicar campo para captcha...")
    input_captcha = driver.find_element(By.CSS_SELECTOR,'[id="tbBuscador:idFormBuscarProceso:codigoCaptcha"][name="tbBuscador:idFormBuscarProceso:codigoCaptcha"][type="text"]').click()
    ActionChains(driver).send_keys(rpta).perform()
    time.sleep(2)
    
    print("buscando info...")
    elemento_buscar = driver.find_element(By.CSS_SELECTOR,'[id="tbBuscador:idFormBuscarProceso:btnBuscarSel"][name="tbBuscador:idFormBuscarProceso:btnBuscarSel"]').click()
    time.sleep(1)
    
    try:
        elemento = driver.find_element(By.CSS_SELECTOR, '[class="ui-growl-message"]')
    except NoSuchElementException:
        print("BÚSQUEDA EJECUTADA CORRECTAMENTE")
        break
    print("Lectura incorrecta, nuevo intento...")
    i = i + 1
    time.sleep(1)


-------------------------------
Lectura_ocr_ 1 :  r2h7p
ubicar campo para captcha...
buscando info...
BÚSQUEDA EJECUTADA CORRECTAMENTE


### 4. BUCLE

#### 4.1. OBTENER LISTA CONCURSOS

In [16]:
tabla = driver.find_element(By.CSS_SELECTOR,'[id="tbBuscador:idFormBuscarProceso:dtProcesos_data"]')
filas = tabla.find_elements(By.TAG_NAME, 'tr')
# Itera en filas
concursos = []
for fila in filas:
    concurso = {
        'num': None,
        'fecha_pub': None,
        'nomenclatura': None,
        'descripcion': None
    }
    celdas = fila.find_elements(By.TAG_NAME, 'td')
    # Iterar en celdas
    i = 0
    for celda in celdas:
        if i == 0:
            num = celda.text
        if i == 1:
            hosp = celda.text
        if i == 2:
            fecha_hora = celda.text
        if i == 3:
            nomenclatura = celda.text
        if i == 6:
            descripcion = celda.text
            #criterio palabra especialización
            if "especializ" in descripcion.lower():
                concurso['num'] = num
                concurso['hospital'] = hosp
                concurso['fecha_pub'] = fecha_hora
                concurso['nomenclatura'] = nomenclatura
                concurso['descripcion'] = descripcion
                concursos.append(concurso)
        i=i+1

#### 4.2. GUARDAR INFO Y ARCHIVOS

In [17]:
import shutil

In [18]:
ruta_descargas = r'C:\Users\Bianca\Downloads'
ruta_destino = r'D:\programitas\proyecto_licitaciones'

In [19]:
for concurso in concursos:
    print("----------------------------------------------------")
    print(concurso['num'])
    print(concurso['nomenclatura'])
    nombre_carpeta = concurso['nomenclatura']
    ruta_carpeta = os.path.join(ruta_destino, nombre_carpeta)
    os.makedirs(ruta_carpeta, exist_ok=True)
    print("carpeta creada")
    
    #clic icono para ver detalles
    pos = int(concurso['num']) - 1
    id_img = 'tbBuscador:idFormBuscarProceso:dtProcesos:'+ str(pos) +':grafichaSel'
    img_concurso = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[id="'+id_img+'"]'))).click()
    print("accediendo al concurso...")
    
    #extración fecha límite
    tb_cron = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[id="tbFicha:dtCronograma_data"]')))
    tr_cron = tb_cron.find_elements(By.TAG_NAME, 'tr')
    i = 0
    for fila_cron in tr_cron:
        celdas_cron = fila_cron.find_elements(By.TAG_NAME, 'td')
        if i == 1:
            j = 0
            for celda_cron in celdas_cron:
                if j == 2:
                    concurso['fecha_lim_registro'] = celda_cron.text
                    print("     dato fecha_lim_registro agregado")
                    break
                j = j+ 1
        i = i + 1
    
    #Crear archivo detalles
    nombre_archivo = ruta_carpeta+"\detalles_"+nomenclatura+".txt"
    with open(nombre_archivo, 'w') as archivo:
        archivo.write("Hopital: "+concurso['hospital']+"\n"
                      +"Nomenclatura: "+concurso['nomenclatura']+"\n"
                      +"Fecha de publicación: "+concurso['fecha_pub']+"\n"
                      +"Descripción: "+concurso['descripcion']+"\n"
                      +"Fecha límite de registro: "+concurso['fecha_lim_registro'])
    print("Se ha creado el archivo")
    
    #extensión de cantidad elementos en lista
    tb_docs = driver.find_element(By.CSS_SELECTOR,'[id="tbFicha:j_idt359"]')
    rangos = tb_docs.find_element(By.CLASS_NAME,"ui-paginator-current").text
    last_pag = int(rangos[-1])
    if last_pag != 1:
        bloq_pags = driver.find_element(By.CSS_SELECTOR,'[id="tbFicha:dtDocumentos_paginator_bottom"]')
        op_list_cant = bloq_pags.find_element(By.CSS_SELECTOR,'[value="15"]').click()
        time.sleep(3)
    
    #descarga de archivos
    t_body = driver.find_element(By.CSS_SELECTOR,'[id="tbFicha:dtDocumentos_data"]')
    tr_docs = t_body.find_elements(By.TAG_NAME, 'tr')
    i = 0
    print("Archivos: ")
    for fila_docs in tr_docs:
        celdas_docs = fila_docs.find_elements(By.TAG_NAME, 'td')
        j = 0
        for celda_docs in celdas_docs:
            if j == 2: 
                print("     " + celda_docs.text)
            if j == 3:
                identif = "tbFicha:dtDocumentos:"+str(i)+":j_idt381"
                a_docs = driver.find_element(By.CSS_SELECTOR,'[id="'+identif+'"]').click()
                print("        descargando...")
                time.sleep(3)
                
                archivo_descargado = 'crdownload'
                while archivo_descargado[-10:] == 'crdownload':
                    archivo_descargado = max([os.path.join(ruta_descargas, f) for f in os.listdir(ruta_descargas)], key=os.path.getctime)
                    time.sleep(2)
                shutil.move(archivo_descargado, ruta_carpeta)
                print("        archivo movido")
            j = j+ 1
        i = i + 1
    btn_regresar = driver.find_element(By.CSS_SELECTOR,'[id="tbFicha:j_idt21"]').click()
    time.sleep(2)
    
print("FIN")

----------------------------------------------------
1
AS-SM-87-2023-HNHU-1
carpeta creada
accediendo al concurso...
     dato fecha_lim_registro agregado
Se ha creado el archivo
Archivos: 
     Resumen ejecutivo
        descargando...
        archivo movido
     Bases Administrativas
        descargando...
        archivo movido
     Pliego de absolución de consultas y observaciones
        descargando...
        archivo movido
     Bases Integradas
        descargando...
        archivo movido
----------------------------------------------------
5
DIRECTA-PROC-16-2023-HNHU-1
carpeta creada
accediendo al concurso...
     dato fecha_lim_registro agregado
Se ha creado el archivo
Archivos: 
     Bases Administrativas
        descargando...
        archivo movido
     Resumen ejecutivo
        descargando...
        archivo movido
     Documentos de Presentación de Propuestas
        descargando...
        archivo movido
     Documentos de Otorgamiento de Buena Pro
        descargando...
